In [3]:
!pip install pymysql
!pip install sqlalchemy

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


You should consider upgrading via the 'c:\users\wkdwoo\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\wkdwoo\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [75]:
!pip install fake_useragent

Using legacy 'setup.py install' for fake-useragent, since package 'wheel' is not installed.
    Running setup.py install for fake-useragent: started
    Running setup.py install for fake-useragent: finished with status 'done'


You should consider upgrading via the 'c:\users\wkdwoo\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
import OpenDartReader
import FinanceDataReader as fdr
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import time
from pathlib import Path

api_key = '85c2367fdad752f376b3947ccdf9998092765f0e'  # OpenDart API KEY

In [6]:
db_connection_str = 'mysql+pymysql://[db유저이름]:[db password]@[host address]/[db name]'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'host address' ([Errno 11001] getaddrinfo failed)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [9]:
dart = OpenDartReader(api_key)

In [10]:
symbols = fdr.StockListing("KRX")
symbols = list(symbols['Symbol'])

In [11]:
symbols[:3]

['060310', '095570', '006840']

In [12]:
# '11013' = 1분기보고서, '11012' = 반기보고서, '11014' = 3분기보고서, '11011' = 사업보고서
reprt_code = ['11013', '11012', '11014', '11011']

In [3]:
temp = dart.finstate_all('060310', 2021)
temp
temp[temp.sj_nm == '현금흐름표'].iloc[0]['thstrm_amount']

AttributeError: 'NoneType' object has no attribute 'sj_nm'

In [48]:
temp = dart.finstate_all('095570', 2021)
temp
temp[(temp.sj_nm == '현금흐름표') & (
        (temp.account_nm == '영업활동으로 인한 현금흐름') | (temp.account_nm == '영업활동 현금흐름'))]

'17725957273'

In [42]:
temp = dart.finstate_all('006840', 2021)
temp
temp[temp.sj_nm == '현금흐름표'].iloc[0]['thstrm_amount']

AttributeError: 'NoneType' object has no attribute 'sj_nm'

In [62]:
str(type(dart.finstate_all('006840', 2015, reprt_code='11014', fs_div='CFS')))

"<class 'NoneType'>"

In [28]:
str(type(temp))

"<class 'NoneType'>"

In [13]:
sample = symbols[1]

In [14]:
type(sample)

str

In [15]:
test = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

In [16]:
# 데이터를 얻기위한 반복문 시작
for symbol in symbols[:3]:
    all_finance_statements = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])
    # 데이터 초기화. 정리된 데이터를 모으는 곳이며 얻고자 하는 데이터 이름을
    # column 화 시킴.  빈 index 를 넣으면 추가가 안되므로 임으로 하나 넣자.

    # 정리 완료된 파일 저장하기 위한 경로 및 이름. result_종목이름.xlsx 형태로 저장된다.
    filePath = 'C:/workspace/capstone-finance/finance-data/data/'
    Path(filePath).mkdir(exist_ok=True)
    fileName = f'2022_1_KRX.csv'
    symbol = str(symbol)
    finance_statement = pd.DataFrame(columns=['종목코드', '유동자산', '부채총계', '자본총계', '매출액', '매출총이익', '영업이익',
                            '당기순이익', '영업활동현금흐름', '잉여현금흐름'], index=['1900-01-01'])

    for i in range(2015, 2022):  # OpenDart는 2015년부터 정보를 제공한다.

        # 더미 리스트 초기화. 1 ~ 4 분기 데이터를 합할 예정이므로 4 크기 만큼의 리스트 선언.
        current_assets = [0, 0, 0, 0]  # 유동자산
        liabilities = [0, 0, 0, 0]  # 부채총계
        equity = [0, 0, 0, 0]  # 자본총계
        revenue = [0, 0, 0, 0]  # 매출액
        gross_marin = [0, 0, 0, 0]  # 매출총이익 #grossMargin
        EBIT = [0, 0, 0, 0]  # 영업이익 #EBIT
        net_income = [0, 0, 0, 0]  # 당기순이익
        cfo = [0, 0, 0, 0]  # 영업활동현금흐름
        cfi = [0, 0, 0, 0]  # 투자활동현금흐름
        fcf = [0, 0, 0, 0]  # 잉여현금흐름 : 편의상 영업활동 - 투자활동 현금흐름으로 계산

        for j, k in enumerate(reprt_code):
            df1 = pd.DataFrame()  # Raw Data

            # 타입이 NoneType 이 아니면 읽어온다.
            if str(type(dart.finstate_all(symbol, i, reprt_code=k, fs_div='CFS'))) != "<class 'NoneType'>":
                df1 = df1.append(dart.finstate_all(symbol, i, reprt_code=k, fs_div='CFS'))
                # 재무상태표 부분
                condition_1 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '유동자산')  # 유동자산
                condition_2 = (df1.sj_nm == '재무상태표') & (df1.account_nm == '부채총계')  # 부채총계
                condition_3 = (df1.sj_nm == '재무상태표') & ((df1.account_nm == '자본총계') | (df1.account_nm == '반기말자본') | (
                        df1.account_nm == '3분기말자본') | (df1.account_nm == '분기말자본') | (
                                                                df1.account_nm == '1분기말자본'))  #자본총계
                # 손익계산서 부분
                condition_4 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '매출액') | (df1.account_nm == '수익(매출액)'))
                condition_5 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (df1.account_nm == '매출총이익')
                condition_6 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '영업이익(손실)') | (df1.account_nm == '영업이익'))
                condition_7 = ((df1.sj_nm == '손익계산서') | (df1.sj_nm == '포괄손익계산서')) & (
                        (df1.account_nm == '당기순이익(손실)') | (df1.account_nm == '당기순이익') |
                        (df1.account_nm == '분기순이익') | (df1.account_nm == '분기순이익(손실)') | (
                                df1.account_nm == '반기순이익') | (df1.account_nm == '반기순이익(손실)') |
                        (df1.account_nm == '연결분기순이익') | (df1.account_nm == '연결반기순이익') | (
                                df1.account_nm == '연결당기순이익') | (df1.account_nm == '연결분기(당기)순이익') | (
                                df1.account_nm == '연결반기(당기)순이익') |
                        (df1.account_nm == '연결분기순이익(손실)'))
                # 현금흐름표 부분
                condition_8 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '영업활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                condition_9 = (df1.sj_nm == '현금흐름표') & (
                        (df1.account_nm == '투자활동으로 인한 현금흐름') | (df1.account_nm == '영업활동 현금흐름'))

                conditions = [condition_1, condition_2, condition_3, condition_4, condition_5, condition_6, condition_7,
                              condition_8]

                stock_code = str(symbol)
                current_assets[j] = 0
                liabilities[j] = 0
                equity[j] = 0
                revenue[j] = 0
                gross_marin[j] = 0
                EBIT[j] = 0
                net_income[j] = 0
                cfo[j] = 0
                cfi[j] = 0

                #for condition in conditions:
                 #   if len(df1[condition]) != 0:
                  #      print(1)


                if len(df1[condition_1]) != 0:
                    current_assets[j] = int(df1.loc[condition_1].iloc[0]['thstrm_amount'])
                if len(df1[condition_2]) != 0:
                    liabilities[j] = int(df1.loc[condition_2].iloc[0]['thstrm_amount'])
                if len(df1[condition_3]) != 0:
                    equity[j] = int(df1.loc[condition_3].iloc[0]['thstrm_amount'])
                if len(df1[condition_4]) != 0:
                    revenue[j] = int(df1.loc[condition_4].iloc[0]['thstrm_amount'])
                if len(df1[condition_5]) != 0:
                    gross_marin[j] = int(df1.loc[condition_5].iloc[0]['thstrm_amount'])
                if len(df1[condition_6]) != 0:
                    EBIT[j] = int(df1.loc[condition_6].iloc[0]['thstrm_amount'])
                if len(df1[condition_7]) != 0:
                    net_income[j] = int(df1.loc[condition_7].iloc[0]['thstrm_amount'])
                if len(df1[condition_8]) != 0:
                    cfo[j] = int(df1[condition_8].iloc[0]['thstrm_amount'])
                if len(df1[condition_9]) != 0:
                    cfi[j] = int(df1[condition_9].iloc[0]['thstrm_amount'])

                fcf[j] = (cfo[j] - cfi[j])

                if k == '11013':  # 1분기.
                    path_string = str(i) + '-03-31'
                elif k == '11012':  # 2분기
                    path_string = str(i) + '-06-30'
                elif k == '11014':  # 3분기
                    path_string = str(i) + '-09-30'
                else:  # 4분기. 1 ~ 3분기 데이터를 더한다음 사업보고서에서 빼야 함
                    path_string = str(i) + '-12-30'
                    revenue[j] = revenue[j] - (revenue[0] + revenue[1] + revenue[2])
                    gross_marin[j] = gross_marin[j] - (gross_marin[0] + gross_marin[1] + gross_marin[2])
                    EBIT[j] = EBIT[j] - (EBIT[0] + EBIT[1] + EBIT[2])
                    net_income[j] = net_income[j] - (net_income[0] + net_income[1] + net_income[2])
                    fcf[j] = fcf[j] - (fcf[0] + fcf[1] + fcf[2])

                # 데이터프레임에 저장하는 부분
                finance_statement.loc[path_string] = [str(stock_code), current_assets[j], liabilities[j], equity[j],
                                        revenue[j], gross_marin[j], EBIT[j], net_income[j], cfo[j], fcf[j]]

                finance_statement.tail()  # 데이터프레임 끝에 저장한다.
                # 너무 잦은 요청이 있을 경우 OpenDart API 측에서 IP 를 차단하니 텀을 둔다.
                time.sleep(0.1)
            else:
                pass
        print(finance_statement)
    all_finance_statements = pd.concat([all_finance_statements, finance_statement[1:]],)
    test.append(all_finance_statements)


    # 원본 dataframe 에도 영향을 끼치게끔 (inplace=True) 첫 행 drop.
    # df2.drop(['1900-01-01'], inplace=True) # 첫 행 drop
test
# 파일 저장. fileName 에 명시된 경로에 각 종목코드별로 다른 이름으로 저장
#all_finance_statements.to_csv(filePath + fileName)

              종목코드         유동자산         부채총계         자본총계          매출액  \
1900-01-01     NaN          NaN          NaN          NaN          NaN   
2015-12-30  060310  16069077218  29420163240  37939808456  23210541648   

                 매출총이익         영업이익        당기순이익 영업활동현금흐름 잉여현금흐름  
1900-01-01         NaN          NaN          NaN      NaN    NaN  
2015-12-30  1635010587  -4157989321  -5599032927        0      0  
              종목코드         유동자산         부채총계         자본총계          매출액  \
1900-01-01     NaN          NaN          NaN          NaN          NaN   
2015-12-30  060310  16069077218  29420163240  37939808456  23210541648   
2016-06-30  060310  13582609176  21382562181  34888339748   6534629920   
2016-09-30  060310  17985342311  24678327990  35061777226   7357350855   
2016-12-30  060310  13426668881  23142629825  34603759371   9850440880   

                 매출총이익         영업이익        당기순이익 영업활동현금흐름 잉여현금흐름  
1900-01-01         NaN          NaN          NaN      NaN    NaN

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [104]:
a = pd.concat([all_finance_statements, finance_statement[1:]],)
#all_finance_statements
a

,종목코드,유동자산,부채총계,자본총계,매출액,매출총이익,영업이익,당기순이익,영업활동현금흐름,잉여현금흐름
1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-30,060310,16069077218,29420163240,37939808456,23210541648,1635010587,-4157989321,-5599032927,0,0
2016-06-30,060310,13582609176,21382562181,34888339748,6534629920,164655511,-1222253624,-1840069164,0,0
2016-09-30,060310,17985342311,24678327990,35061777226,7357350855,63868668,-766080221,-489723045,0,0


In [94]:
finance_statement

,종목코드,유동자산,부채총계,자본총계,매출액,매출총이익,영업이익,당기순이익,영업활동현금흐름,잉여현금흐름
1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-30,006840,969178973579,1593921533934,901593458808,2807147867285,837072464777,106316161081,-37443062232,0,0
2016-03-31,006840,988631258591,1583122119182,923480507913,674546018889,227004863303,54380571530,39242842322,0,0
2016-06-30,006840,983783832774,1561978780722,957297865893,703651872826,210205795074,41925130198,33435136319,0,0
2016-09-30,006840,1026368189738,1582981704199,1000523063990,756067500602,248200163586,72467279677,45853481018,0,0
2016-12-30,006840,1179857738221,1730525757838,1011420744775,787565671300,234627061621,44896723153,14790881151,0,0
2017-03-31,006840,1156717911454,1751152521875,1024648091822,820740707452,250862150870,66723415561,43407295013,0,0
2017-06-30,006840,1142676494202,1710653032430,1059493940990,795495426569,235212592980,54142066193,38874109446,0,0
2017-09-30,006840,1202890814676,1766738408262,1135115218167,865566647573,276364282433,88833747924,77002602585,0,262361994205
2017-12-30,006840,1218481945179,1792720288489,1152064058157,910651828911,301839931936,55942037393,26130010299,0,30150579342


In [78]:
print(symbols)

['060310', '095570', '006840', '054620', '265520', '211270', '152100', '295820', '253150', '253160', '395750', '395760', '278420', '292750', '309210', '333940', '333950', '333960', '333970', '333980', '269530', '251590', '161510', '251600', '289670', '298340', '189400', '419650', '415920', '278620', '269540', '287180', '213630', '332610', '332620', '195970', '238670', '195980', '373530', '256450', '239660', '280920', '266550', '301400', '301410', '227830', '122090', '328370', '301440', '376250', '027410', '282330', '032790', '138930', '001460', '001465', '013720', '001040', '079160', '035760', '311690', '00104K', '000120', '011150', '011155', '001045', '097950', '097955', '051500', '058820', '023460', '056730', '056000', '065770', '083660', '000590', '012030', '404950', '367340', '367360', '016610', '005830', '000990', '000995', '139130', '001530', '000210', '001880', '000215', '375500', '37550K', '068790', '004840', '241520', '155660', '069730', '017940', '245620', '037370', '278990',

In [65]:
print(df1[condition_8], i)

Empty DataFrame
Columns: [rcept_no, reprt_code, bsns_year, corp_code, sj_div, sj_nm, account_id, account_nm, account_detail, thstrm_nm, thstrm_amount, frmtrm_nm, frmtrm_amount, bfefrmtrm_nm, bfefrmtrm_amount, ord, thstrm_add_amount]
Index: [] 2015


In [66]:
len(df1[condition_8])

0

In [69]:
conditions = [condition_1, condition_2, condition_3, condition_4, condition_5, condition_6, condition_7,
                              condition_8]

In [70]:
for condition in conditions:
                    if len(df1[condition]) != 0:
                        print(1)

1
1
1
1


In [93]:
all_finance_statements

,종목코드,유동자산,부채총계,자본총계,매출액,매출총이익,영업이익,당기순이익,영업활동현금흐름,잉여현금흐름
1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
symbols.index('035900')



187